In [9]:
import asyncio
import base64
import json
import os
import pyaudio
import shutil
import websockets
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv("openai.env")

class AudioStreamer:
    def __init__(self):
        self.p = pyaudio.PyAudio()

    def mic_audio_in_callback(self, in_data, frame_count, time_info, status):
        payload = base64.b64encode(in_data).decode("utf-8")

        async def send():
            await self.ws.send(
                json.dumps(
                    {
                        "type": "input_audio_buffer.append",
                        "audio": payload,
                    },
                )
            )

        asyncio.run(send())
        return (None, pyaudio.paContinue)

    async def ws_receive_worker(self):
        async for m in self.ws:
            columns, rows = shutil.get_terminal_size()
            maxl = columns - 5
            print(m if len(m) <= maxl else (m[:maxl] + " ..."))
            evt = json.loads(m)
            if evt["type"] == "session.created":
                print("Connected: say something to GPT-4o")
                self.mic_audio_in.start_stream()
            elif evt["type"] == "response.audio.delta":
                audio = base64.b64decode(evt["delta"])
                self.speaker_audio_out.write(audio)

    async def run(self):
        self.mic_audio_in = self.p.open(
            format=pyaudio.paInt16,
            channels=1,
            rate=24000,
            input=True,
            stream_callback=self.mic_audio_in_callback,
            frames_per_buffer=int(24000 / 100) * 2,  # 20ms of audio
            start=False,
        )

        self.speaker_audio_out = self.p.open(
            format=pyaudio.paInt16,
            channels=1,
            rate=24000,
            output=True,
        )

        self.ws = await websockets.connect(
            uri="wss://api.openai.com/v1/realtime?model=gpt-4o-realtime-preview-2024-10-01",
            extra_headers={
                "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
                "OpenAI-Beta": "realtime=v1",
            },
        )

        asyncio.create_task(self.ws_receive_worker())

        await asyncio.sleep(15 * 60)


In [10]:
# 원래 코드

# if __name__ == "__main__":
#     asyncio.run(AudioStreamer().run())

# 변경 코드 
if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    if loop.is_running():
        # 이미 실행 중인 이벤트 루프가 있는 경우 (인터랙티브 환경에서 사용)
        loop.create_task(AudioStreamer().run())
    else:
        loop.run_until_complete(AudioStreamer().run())



{"type":"session.created","event_id":"event_AG5KoUffHQA3F2YEhdhRg","session ...
Connected: say something to GPT-4o
{"type":"input_audio_buffer.speech_started","event_id":"event_AG5PP8hGw1cgZ ...
{"type":"input_audio_buffer.speech_stopped","event_id":"event_AG5PQXW3U3itL ...
{"type":"input_audio_buffer.committed","event_id":"event_AG5PQn8gf19LL1OiFe ...
{"type":"conversation.item.created","event_id":"event_AG5PQlIQ2bjSBaud8ivqh ...
{"type":"response.created","event_id":"event_AG5PQFkLCBAqrzeTWRNxs","respon ...
{"type":"input_audio_buffer.speech_started","event_id":"event_AG5PQm08fu0es ...
{"type":"response.done","event_id":"event_AG5PQ8CUOSer9ZyfZsnJJ","response" ...
{"type":"input_audio_buffer.speech_stopped","event_id":"event_AG5PRYqxGbGmU ...
{"type":"input_audio_buffer.committed","event_id":"event_AG5PRezWFtl3eMIWwe ...
{"type":"conversation.item.created","event_id":"event_AG5PRdMjMzalWWB67q7Y8 ...
{"type":"response.created","event_id":"event_AG5PRZIhnnu7rBq3K87Bf","respon ...
{"typ

Fatal error on SSL protocol
protocol: <asyncio.sslproto.SSLProtocol object at 0x00000219C82C1340>
transport: <_SelectorSocketTransport closing fd=2112 read=idle write=<idle, bufsize=0>>
Traceback (most recent call last):
  File "c:\Users\rumfox\AppData\Local\Programs\Python\Python312\Lib\asyncio\sslproto.py", line 694, in _write_appdata
    self._do_write()
  File "c:\Users\rumfox\AppData\Local\Programs\Python\Python312\Lib\asyncio\sslproto.py", line 709, in _do_write
    del self._write_backlog[0]
        ~~~~~~~~~~~~~~~~~~~^^^
IndexError: deque index out of range
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\rumfox\AppData\Local\Temp\ipykernel_3752\407605252.py", line 30, in mic_audio_in_callback
    asyncio.run(send())
  File "c:\Users\rumfox\AppData\Local\Programs\Python\Python312\Lib\asyncio\runners.py", line 194, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "c:\Users\rumfox\AppData\Local\Programs\Python\Python312\Lib\asyncio\ru